In [1]:

import torch
from torch_geometric.data import Data
import numpy as np
import torch

def create_edge_index(n):
   # create edge_index tensor
    edge_index = torch.zeros((2, n*n), dtype=torch.long)

    # fill edge_index tensor with connections
    for i in range(n):
        for j in range(n):
            if i != j:
                edge_index[0, i*n+j] = i
                edge_index[1, i*n+j] = j

    # add reverse connections
    edge_index = torch.cat([edge_index, torch.stack([edge_index[1], edge_index[0]], dim=0)], dim=1)
    return edge_index


def graphFrame(frame, df):
    data = df[df['frameIndex'] == frame]
    object_ids = data["id"]
    node_count = object_ids.count()
    edge_indexies = create_edge_index(node_count)
    attributes = data[['category','attributes.crowd','box2d.x1','box2d.x2','box2d.y1','box2d.y2']]
    attributes['attributes.crowd'] = attributes['attributes.crowd'].map({True: 1, False: 0})
    attributes['category'] = attributes['category'].map({'car': 1, 'pedestrian': 0, 3: 3}).fillna(3).astype(int)


    # convert numpy array to tensor
    x = torch.Tensor(attributes.select_dtypes(include=[np.number]).values)
    graphData = Data(x=x, edge_index=edge_indexies)
    return data, graphData, object_ids, attributes, node_count

In [2]:

print(create_edge_index(5))

tensor([[0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 2, 2, 0, 2, 2, 3, 3, 3, 0, 3, 4, 4, 4, 4,
         0, 0, 1, 2, 3, 4, 0, 0, 2, 3, 4, 0, 1, 0, 3, 4, 0, 1, 2, 0, 4, 0, 1, 2,
         3, 0],
        [0, 1, 2, 3, 4, 0, 0, 2, 3, 4, 0, 1, 0, 3, 4, 0, 1, 2, 0, 4, 0, 1, 2, 3,
         0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 2, 2, 0, 2, 2, 3, 3, 3, 0, 3, 4, 4, 4,
         4, 0]])


In [3]:
import pandas as pd

# read in the data from a CSV file
df = pd.read_csv('splitData/0000f77c-62c2a288.csv')
data, graphData, object_ids, attributes, node_count = graphFrame(0, df)

C:\Users\Frank\AppData\Local\Temp\ipykernel_23756\83198924.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  attributes['attributes.crowd'] = attributes['attributes.crowd'].map({True: 1, False: 0})
C:\Users\Frank\AppData\Local\Temp\ipykernel_23756\83198924.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  attributes['category'] = attributes['category'].map({'car': 1, 'pedestrian': 0, 3: 3}).fillna(3).astype(int)


In [4]:
graphData

Data(x=[3, 6], edge_index=[2, 18])

In [5]:
print(data)
print(object_ids)
print(node_count)

   Unnamed: 0                           name          videoName  frameIndex  \
0     2837858  0000f77c-62c2a288-0000001.jpg  0000f77c-62c2a288           0   
1     2837859  0000f77c-62c2a288-0000001.jpg  0000f77c-62c2a288           0   
2     2837860  0000f77c-62c2a288-0000001.jpg  0000f77c-62c2a288           0   

        id    category attributes.crowd attributes.occluded  \
0  20715.0         car            False               False   
1  20716.0  pedestrian            False               False   
2  20717.0  pedestrian            False               False   

  attributes.truncated    box2d.x1    box2d.x2    box2d.y1    box2d.y2  \
0                False  123.834586  236.165414  338.345865  396.541353   
1                False  244.285714  263.233083  346.466165  378.947368   
2                False  257.819549  278.120301  339.699248  378.947368   

   haveVideo  
0       True  
1       True  
2       True  
0    20715.0
1    20716.0
2    20717.0
Name: id, dtype: float64
3


In [6]:
#category,attributes.crowd,box2d.x1,box2d.x2,box2d.y1,box2d.y2
import torch_geometric.nn.models as tgm

lookback = 5#use past 5 frames
hidden_channels = 10# don't know what this is
layers =10
out_channel = 1


m = tgm.GCN(lookback, hidden_channels, layers, out_channel)

In [7]:
def graph_multFrame_test(df, first_frame, end_frame, view_length):
    arr = np.full((end_frame-first_frame-1, view_length), -1)
    
    for x in range(0, end_frame-first_frame+view_length-2):
        graph = x+first_frame#graphFrame(x+first_frame, df)
        
        for y in range(0, 3):
            if((y+x<end_frame-first_frame+view_length-2)and(x+y-(view_length-1)>=0)):
                arr[x+y-(view_length-1)][y] = graph
                
    
    return arr

In [8]:
arr= graph_multFrame_test(0, 0, 10, 3)
print(arr)
arr= graph_multFrame_test(0, 1, 11, 3)
print(arr)

[[ 2  1  0]
 [ 3  2  1]
 [ 4  3  2]
 [ 5  4  3]
 [ 6  5  4]
 [ 7  6  5]
 [ 8  7  6]
 [ 9  8  7]
 [10  9  8]]
[[ 3  2  1]
 [ 4  3  2]
 [ 5  4  3]
 [ 6  5  4]
 [ 7  6  5]
 [ 8  7  6]
 [ 9  8  7]
 [10  9  8]
 [11 10  9]]


In [9]:
def graph_multFrame(df, first_frame, end_frame, view_length):
    arr = np.empty((end_frame-first_frame-1, view_length), dtype = Data)
    
    for x in range(0, end_frame-first_frame+view_length-2):
        frame = x+first_frame
        graph = graphFrame(frame, df)[1]
        
        #print( graph)
        
        for y in range(0, 3):
            if((y+x<end_frame-first_frame+view_length-2)and(x+y-(view_length-1)>=0)):
                arr[x+y-(view_length-1)][y] = graph
                
    
    return arr

In [10]:
splitData = graph_multFrame(df, 0, 10, lookback)


C:\Users\Frank\AppData\Local\Temp\ipykernel_23756\83198924.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  attributes['attributes.crowd'] = attributes['attributes.crowd'].map({True: 1, False: 0})
C:\Users\Frank\AppData\Local\Temp\ipykernel_23756\83198924.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  attributes['category'] = attributes['category'].map({'car': 1, 'pedestrian': 0, 3: 3}).fillna(3).astype(int)
C:\Users\Frank\AppData\Local\Temp\ipykernel_23756\83198924.py:30: SettingWithCopyWarning: 